In [1]:
import numpy as np
import pandas as pd
import scipy.stats

pd.options.mode.chained_assignment = None
from matplotlib import pyplot as plt, rcParams
# import cv2
import seaborn as sns

sns.set(style="white", context="paper")
from cycler import cycler
import os, sys
import glob
from datetime import datetime, timedelta
from itertools import combinations
import base64
from PIL import Image
from io import BytesIO as _BytesIO
import requests
import json
import pickle
from datetime import datetime
from IPython.display import display, Markdown, Latex
from sklearn.metrics import *
import collections
from copy import deepcopy
import traceback
# import plotly
# from pandas_profiling import ProfileReport

pd.options.display.max_columns = None


def printm(s): return display(Markdown(s))

/home/prasoon/anaconda3/envs/edusense/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.25.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Figure out how many sessions are there, and how many frames for each session

In [8]:
track_analysis_meta_cache = '../cache/analysis_all/meta_info_all'
os.makedirs(track_analysis_meta_cache,exist_ok=True)

In [9]:
base_dir = '/mnt/ci-nas-cache/edulyzeV2/pose_face_gaze_emb/'
frame_file_data = {}
for course_idx, course_dir in enumerate(glob.glob(f"{base_dir}/*")):
    course_name = course_dir.split("/")[-1]
    course_cache_file = f"{track_analysis_meta_cache}/{course_name}"
    if os.path.exists(course_cache_file):
        frame_file_data[course_name] = pickle.load(open(course_cache_file,"rb"))
        continue
    frame_file_data[course_name]={}
        
    for session_idx, session_dir in enumerate(glob.glob(f"{course_dir}/*")):
        session_name = session_dir.split("/")[-1]
        frame_file_data[course_name][session_name] = {}
        frame_files = glob.glob(f"{session_dir}/*")
        frame_file_names = [xr.split("/")[-1] for xr in frame_files]
        if 'end.pb' in frame_file_names:
            frame_file_data[course_name][session_name]['is_completed']=True
        else:
            frame_file_data[course_name][session_name]['is_completed']=False            
        frame_ids = [int(xr.split(".")[0]) for xr in frame_file_names if not (xr=='end.pb')]
        frame_file_data[course_name][session_name]['frame_ids'] = sorted(frame_ids)
        frame_file_data[course_name][session_name]['dir_location'] = session_dir
        print(f"Got metadata for course: {course_idx}-{course_name}, session:{session_idx}-{session_name}")
    pickle.dump(frame_file_data[course_name],open(course_cache_file,"wb")) 
        
        
        

Got metadata for course: 0-05410A, session:0-classinsight-cmu_05410A_ghc_4301_201902181500-front
Got metadata for course: 0-05410A, session:1-classinsight-cmu_05410A_ghc_4301_201902251500-front
Got metadata for course: 0-05410A, session:2-classinsight-cmu_05410A_ghc_4301_201904081500-front
Got metadata for course: 0-05410A, session:3-classinsight-cmu_05410A_ghc_4301_201904151500-front
Got metadata for course: 0-05410A, session:4-classinsight-cmu_05410A_ghc_4301_201904221500-front
Got metadata for course: 1-21127J, session:0-classinsight-cmu_21127J_ghc_4102_201903190930-front
Got metadata for course: 1-21127J, session:1-classinsight-cmu_21127J_ghc_4102_201903260930-front
Got metadata for course: 1-21127J, session:2-classinsight-cmu_21127J_ghc_4102_201904160930-front
Got metadata for course: 1-21127J, session:3-classinsight-cmu_21127J_ghc_4102_201904230930-front
Got metadata for course: 1-21127J, session:4-classinsight-cmu_21127J_ghc_4102_201904300930-front
Got metadata for course: 2-057

In [10]:
# course_name, session_name, is_completed, min_frame, max_frame_number, num_frames, session_duration_mins, 
# Next step: Get how many sessions are completed, and 
session_infos = []
for course_name in frame_file_data:
    session_meta_data = {'course':course_name}
    for session_name in frame_file_data[course_name]:
        print(session_name)
        session_meta_data['session']=session_name.split("-front")[0]
        try:
            session_meta_data['start_time'] = pd.to_datetime(session_meta_data['session'].split("_")[-1], format="%Y%m%d%H%M")
        except:
            session_meta_data['start_time'] = pd.to_datetime(session_meta_data['session'].split("_")[-1], format="%Y%m%d%H%M%S")
        session_meta_data['start_hour'] = session_meta_data['start_time'].strftime('%H:%M')
        session_meta_data['classroom'] = "_".join(session_meta_data['session'].split("_")[-3:-1])
        meta_data = frame_file_data[course_name][session_name]
        session_meta_data['completed'] = meta_data['is_completed']
        print(meta_data['frame_ids'])
        session_meta_data['num_frames'] = len(meta_data['frame_ids'])
        if len(meta_data['frame_ids']) > 0:
            session_meta_data['max_frame'] = max(meta_data['frame_ids'])
            session_meta_data['duration_in_mins'] = max(meta_data['frame_ids'])//(15*60)
        session_infos.append(session_meta_data)
        # print(course_name, session_name)
df_session_info = pd.DataFrame(session_infos)   

classinsight-cmu_05410A_ghc_4301_201902181500-front
[0, 3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48, 51, 54, 57, 60, 63, 66, 69, 72, 75, 78, 81, 84, 87, 90, 93, 96, 99, 102, 105, 108, 111, 114, 117, 120, 123, 126, 129, 132, 135, 138, 141, 144, 147, 150, 153, 156, 159, 162, 165, 168, 171, 174, 177, 180, 183, 186, 189, 192, 195, 198, 201, 204, 207, 210, 213, 216, 219, 222, 225, 228, 231, 234, 237, 240, 243, 246, 249, 252, 255, 258, 261, 264, 267, 270, 273, 276, 279, 282, 285, 288, 291, 294, 297, 300, 303, 306, 309, 312, 315, 318, 321, 324, 327, 330, 333, 336, 339, 342, 345, 348, 351, 354, 357, 360, 363, 366, 369, 372, 375, 378, 381, 384, 387, 390, 393, 396, 399, 402, 405, 408, 411, 414, 417, 420, 423, 426, 429, 432, 435, 438, 441, 444, 447, 450, 453, 456, 459, 462, 465, 468, 471, 474, 477, 480, 483, 486, 489, 492, 495, 498, 501, 504, 507, 510, 513, 516, 519, 522, 525, 528, 531, 534, 537, 540, 543, 546, 549, 552, 555, 558, 561, 564, 567, 570, 573, 576, 579, 582, 585, 588, 

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [11]:
session_infos[-1]

{'course': '05410B',
 'session': 'classinsight-cmu_05410B_ghc_4211_201904081500',
 'start_time': Timestamp('2019-04-08 15:00:00'),
 'start_hour': '15:00',
 'classroom': 'ghc_4211',
 'completed': False,
 'num_frames': 11741,
 'max_frame': 35220,
 'duration_in_mins': 39}

In [12]:
df_session_info.head()

,course,session,start_time,start_hour,classroom,completed,num_frames,max_frame,duration_in_mins
0,05410A,classinsight-cmu_05410A_ghc_4301_201904221500,2019-04-22 15:00:00,15:00,ghc_4301,True,23998,71991,79
1,05410A,classinsight-cmu_05410A_ghc_4301_201904221500,2019-04-22 15:00:00,15:00,ghc_4301,True,23998,71991,79
2,05410A,classinsight-cmu_05410A_ghc_4301_201904221500,2019-04-22 15:00:00,15:00,ghc_4301,True,23998,71991,79
3,05410A,classinsight-cmu_05410A_ghc_4301_201904221500,2019-04-22 15:00:00,15:00,ghc_4301,True,23998,71991,79
4,05410A,classinsight-cmu_05410A_ghc_4301_201904221500,2019-04-22 15:00:00,15:00,ghc_4301,True,23998,71991,79


In [13]:
df_session_info = df_session_info[df_session_info.completed]
df_session_info['course_num'] = df_session_info.course.apply(lambda x: int(x[:5]) if len(x)>5 else int(x[:4]))
df_session_info['course_num'].value_counts()

5410     5
21127    5
Name: course_num, dtype: int64

# Include course meta info 

In [14]:
df_course_info = pd.read_csv('course_descriptions_1.csv')
df_course_info2 = pd.read_csv('course_descriptions_2.csv')
df_course_info = pd.concat([df_course_info,df_course_info2],ignore_index=True)
df_course_info = df_course_info[~df_course_info.course_name.isnull()]
df_course_info.info()
df_course_info.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 63 entries, 0 to 64
Data columns (total 19 columns):
 #   Column                                               Non-Null Count  Dtype  
---  ------                                               --------------  -----  
 0   S.No                                                 63 non-null     int64  
 1   Semester                                             51 non-null     object 
 2   Division                                             51 non-null     object 
 3   Dept                                                 51 non-null     object 
 4   course_num                                           63 non-null     int64  
 5   course_name                                          63 non-null     object 
 6   Course Level                                         51 non-null     object 
 7   Hrs Per Week                                         51 non-null     float64
 8   course_category                                      62 non-null     obj

,S.No,Semester,Division,Dept,course_num,course_name,Course Level,Hrs Per Week,course_category,description,Interest in student learning,Clearly explain course requirements,Clear learning objectives & goals,Instructor provides feedback to students to improve,Demonstrate importance of subject matter,Explains subject matter of course,Show respect for all students,Overall teaching rate,Overall course rate
0,1,Fall,Carnegie Institute of Technology,CHE,6705,ADV CHE THRMODYNMCS,Graduate,8.16,theory,Advanced application of the general thermodyna...,4.42,4.63,4.74,4.05,4.58,4.63,4.95,4.47,4.37
1,2,Fall,Mellon College of Science,CMY,9214,PHYSICAL CHEMISTRY,Undergraduate,6.50,theory,This is a one-semester course intended primari...,5.00,4.50,4.50,4.50,5.00,4.50,5.00,4.50,4.50
2,3,Fall,Mellon College of Science,CMY,9403,CMY ADDIC,Undergraduate,5.75,theory,What makes us need something so much that it e...,5.00,4.75,5.00,5.00,5.00,4.88,5.00,5.00,4.88
3,4,Fall,Carnegie Institute of Technology,CEE,12411,PRJ MANGMT CONSTRCTN,Undergraduate,7.46,applied,"Through planning and management, and optimizat...",4.34,3.98,4.05,3.89,4.48,4.02,4.69,3.98,3.72
4,5,Fall,School of Computer Science,CS,15251,GRT IDEAS THERTCL CS,Undergraduate,14.27,theory,This course is about how to use theoretical id...,4.71,4.69,4.72,4.53,4.72,4.72,4.81,4.67,4.72


In [15]:
df_session_info = pd.merge(df_session_info, df_course_info, on=['course_num'])

In [16]:
df_session_info.head()

,course,session,start_time,start_hour,classroom,completed,num_frames,max_frame,duration_in_mins,course_num,S.No,Semester,Division,Dept,course_name,Course Level,Hrs Per Week,course_category,description,Interest in student learning,Clearly explain course requirements,Clear learning objectives & goals,Instructor provides feedback to students to improve,Demonstrate importance of subject matter,Explains subject matter of course,Show respect for all students,Overall teaching rate,Overall course rate
0,05410A,classinsight-cmu_05410A_ghc_4301_201904221500,2019-04-22 15:00:00,15:00,ghc_4301,True,23998,71991,79,5410,8,NaN,NaN,NaN,User-Centered Research and Evaluation,NaN,NaN,theory,This course provides an overview and introduct...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,05410A,classinsight-cmu_05410A_ghc_4301_201904221500,2019-04-22 15:00:00,15:00,ghc_4301,True,23998,71991,79,5410,8,NaN,NaN,NaN,User-Centered Research and Evaluation,NaN,NaN,theory,This course provides an overview and introduct...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,05410A,classinsight-cmu_05410A_ghc_4301_201904221500,2019-04-22 15:00:00,15:00,ghc_4301,True,23998,71991,79,5410,8,NaN,NaN,NaN,User-Centered Research and Evaluation,NaN,NaN,theory,This course provides an overview and introduct...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,05410A,classinsight-cmu_05410A_ghc_4301_201904221500,2019-04-22 15:00:00,15:00,ghc_4301,True,23998,71991,79,5410,8,NaN,NaN,NaN,User-Centered Research and Evaluation,NaN,NaN,theory,This course provides an overview and introduct...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,05410A,classinsight-cmu_05410A_ghc_4301_201904221500,2019-04-22 15:00:00,15:00,ghc_4301,True,23998,71991,79,5410,8,NaN,NaN,NaN,User-Centered Research and Evaluation,NaN,NaN,theory,This course provides an overview and introduct...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
df_session_info[df_session_info.course=='21127L']

,course,session,start_time,start_hour,classroom,completed,num_frames,max_frame,duration_in_mins,course_num,S.No,Semester,Division,Dept,course_name,Course Level,Hrs Per Week,course_category,description,Interest in student learning,Clearly explain course requirements,Clear learning objectives & goals,Instructor provides feedback to students to improve,Demonstrate importance of subject matter,Explains subject matter of course,Show respect for all students,Overall teaching rate,Overall course rate


In [18]:
df_course_info = df_session_info.groupby(['course','Semester','Course Level','course_category','Division','start_hour']).agg({
    'completed':'count',
    'duration_in_mins':['min','mean','max'],
    'num_frames':['min','mean','max'],}).reset_index()
df_course_info.columns = ['course','Semester','Course Level','course_category','Division','start_hour','num_sessions','dur_min','dur_mean','dur_max','frame_count_min','frame_count_mean','frame_count_max']
df_course_info= df_course_info[df_course_info.dur_mean > 25]
df_course_info.sort_values(by='start_hour',ascending=False).reset_index(drop=True)

,course,Semester,Course Level,course_category,Division,start_hour,num_sessions,dur_min,dur_mean,dur_max,frame_count_min,frame_count_mean,frame_count_max
0,21127J,Fall,Undergraduate,theory,Mellon College of Science,09:30,5,49,49.0,49,14999,14999.0,14999
1,21127J,Spring,Undergraduate,theory,Mellon College of Science,09:30,5,49,49.0,49,14999,14999.0,14999
2,21127J,Summer,Undergraduate,theory,Mellon College of Science,09:30,5,49,49.0,49,14999,14999.0,14999


In [19]:
# 15 courses: 1.5 hours
# 18 courses: 1 hour
# 2 courses: half hour


# Cache single session tracking for id consistency

In [20]:
# sample_course = '17214B'
# sample_session = 'classinsight-cmu_17214B_ph_a21_201905011030-front'
# sample_session_dir = frame_file_data[sample_course][sample_session]['dir_location']
# sample_frame_ids = frame_file_data[sample_course][sample_session]['frame_ids']
# sample_session_dir

In [21]:
# session_tracking_ids = {}
# for frame_id in sample_frame_ids:
#     frame_data = pickle.load(open(f'{sample_session_dir}/{frame_id}.pb','rb'))
#     frame_tracking_ids = [xr['track_id'] for xr in frame_data[1]]
#     # print(frame_id, frame_tracking_ids)
#     session_tracking_ids[frame_id] = {int(xr):1 for xr in frame_tracking_ids}
# df_session_ids = pd.DataFrame.from_dict(session_tracking_ids)

In [22]:
# df_session_ids.head()

In [23]:
# np.unique(np.nansum(df_session_ids.values,axis=1) // 5, return_counts=True)

In [24]:
session_tracking_cache = '../cache/analysis_tracking/session_tracking_info'


In [19]:
unfinished_sessions = []
for course_idx, course in enumerate(frame_file_data):
    for session_idx, session_id in enumerate(frame_file_data[course]):
        session_tracking_cache_file = f"{session_tracking_cache}/{session_id}.pb"
        try:
            if not os.path.exists(session_tracking_cache_file):
                session_dir = frame_file_data[course][session_id]['dir_location']
                frame_ids = frame_file_data[course][session_id]['frame_ids']
                session_tracking_ids = {}
                for frame_id in frame_ids:
                    frame_data = pickle.load(open(f'{session_dir}/{frame_id}.pb','rb'))
                    frame_tracking_ids = [xr['track_id'] for xr in frame_data[1]]
                    # print(frame_id, frame_tracking_ids)
                    session_tracking_ids[frame_id] = {int(xr):1 for xr in frame_tracking_ids}
                df_session_ids = pd.DataFrame.from_dict(session_tracking_ids)
                pickle.dump(df_session_ids, open(session_tracking_cache_file,'wb'))
                print(f"Got tracking info for session: {course_idx}-{course}, session:{session_idx}-{session_id}")
            else:
                ...
                # print(f"FILE EXISTS: tracking info for session: {course_idx}-{course}, session:{session_idx}-{session_id}")
        except:
            print(f"ERROR: Unable to get session tracking for: {course_idx}-{course}, session:{session_idx}-{session_id}")
            unfinished_sessions.append((course, session_id))
            print(traceback.format_exc())
        

Got tracking info for session: 45-05772A, session:1-classinsight-cmu_05772A_ghc_4301_201902051030-front
Got tracking info for session: 45-05772A, session:3-classinsight-cmu_05772A_ghc_4301_201902121030-front
Got tracking info for session: 46-09105A, session:0-classinsight-cmu_09105A_ghc_4102_201901291830-front
Got tracking info for session: 46-09105A, session:11-classinsight-cmu_09105A_ghc_4102_201903071830-front
Got tracking info for session: 46-09105A, session:12-classinsight-cmu_09105A_ghc_4102_201903121830-front
Got tracking info for session: 46-09105A, session:15-classinsight-cmu_09105A_ghc_4102_201903211830-front
Got tracking info for session: 46-09105A, session:27-classinsight-cmu_09105A_ghc_4102_201905021830-front
Got tracking info for session: 47-09105C, session:1-classinsight-cmu_09105C_ghc_5222_201905021830-front
Got tracking info for session: 47-09105C, session:3-classinsight-cmu_09105C_ghc_5222_201902051830-front
Got tracking info for session: 47-09105C, session:5-classins

# Setup tracking id experiment by random selection of courses, sessions and snippets

In [26]:
df_session_info.head()

,course,session,start_time,start_hour,classroom,completed,num_frames,max_frame,duration_in_mins,course_num,S.No,Semester,Division,Dept,course_name,Course Level,Hrs Per Week,course_category,description,Interest in student learning,Clearly explain course requirements,Clear learning objectives & goals,Instructor provides feedback to students to improve,Demonstrate importance of subject matter,Explains subject matter of course,Show respect for all students,Overall teaching rate,Overall course rate
0,17214B,classinsight-cmu_17214B_ph_a21_201905011030,2019-05-01 10:30:00,10:30,ph_a21,True,14999,44994.0,49.0,17214,9,Fall,School of Computer Science,ISR,PRIN 0-0 SFTWR CONST,Undergraduate,14.51,applied,Software engineers today are less likely to de...,4.27,4.24,4.38,4.04,4.36,4.16,4.40,4.13,4.13
1,17214B,classinsight-cmu_17214B_ph_a21_201905011030,2019-05-01 10:30:00,10:30,ph_a21,True,14999,44994.0,49.0,17214,28,Spring,School of Computer Science,SE,PRIN 0-0 SFTWR CONST,Undergraduate,13.09,applied,Software engineers today are less likely to de...,4.39,4.18,4.48,3.88,4.51,4.38,4.86,4.26,4.26
2,17214B,classinsight-cmu_17214B_ph_a21_201905011030,2019-05-01 10:30:00,10:30,ph_a21,True,14999,44994.0,49.0,17214,9,Fall,School of Computer Science,ISR,PRIN 0-0 SFTWR CONST,Undergraduate,14.51,applied,Software engineers today are less likely to de...,4.27,4.24,4.38,4.04,4.36,4.16,4.40,4.13,4.13
3,17214B,classinsight-cmu_17214B_ph_a21_201905011030,2019-05-01 10:30:00,10:30,ph_a21,True,14999,44994.0,49.0,17214,28,Spring,School of Computer Science,SE,PRIN 0-0 SFTWR CONST,Undergraduate,13.09,applied,Software engineers today are less likely to de...,4.39,4.18,4.48,3.88,4.51,4.38,4.86,4.26,4.26
4,17214B,classinsight-cmu_17214B_ph_a21_201905011030,2019-05-01 10:30:00,10:30,ph_a21,True,14999,44994.0,49.0,17214,9,Fall,School of Computer Science,ISR,PRIN 0-0 SFTWR CONST,Undergraduate,14.51,applied,Software engineers today are less likely to de...,4.27,4.24,4.38,4.04,4.36,4.16,4.40,4.13,4.13


In [24]:
# randomly select 10 courses.
np.random.seed(43)
prefiltered_courses = df_session_info.course.drop_duplicates().sample(n=10)
print(f"Prefiltered courses:{prefiltered_courses}")

# select 5 sessions per course 
df_prefiltered_sessions = pd.DataFrame()
for course in prefiltered_courses.values:
    if df_session_info[df_session_info.course==course].shape[0]>5:
        df_prefiltered_sessions.append(df_session_info[df_session_info.course==course].sample(n=5))
df_prefiltered_sessions

Prefiltered courses:541     79388A
127     05410B
597     12411B
753     36200S
155     21127J
676     09105D
239     21127L
851     76270S
619     09105A
1120    17346A
Name: course, dtype: object


""


In [25]:
df_prefiltered_sessions

""
